In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from numpy.polynomial.polynomial import polyfit
from sklearn.linear_model import LogisticRegression
import seaborn as sns
import statsmodels.stats.tests.test_influence
from sklearn.feature_selection import RFE
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

## __1 - Business Problem__  
___Whether the client has subscribed a term deposit or not Binomial ("yes" or "no")___  

## __2 - Data collection and description__ 

In [10]:
df = pd.read_csv("../DATASCIENCE/bank-full (1).csv")

In [11]:
df1 = df.copy()

In [12]:
df1

,"age;""job"";""marital"";""education"";""default"";""balance"";""housing"";""loan"";""contact"";""day"";""month"";""duration"";""campaign"";""pdays"";""previous"";""poutcome"";""y"""
0,"58;""management"";""married"";""tertiary"";""no"";2143..."
1,"44;""technician"";""single"";""secondary"";""no"";29;""..."
2,"33;""entrepreneur"";""married"";""secondary"";""no"";2..."
3,"47;""blue-collar"";""married"";""unknown"";""no"";1506..."
4,"33;""unknown"";""single"";""unknown"";""no"";1;""no"";""n..."
...,...
45206,"51;""technician"";""married"";""tertiary"";""no"";825;..."
45207,"71;""retired"";""divorced"";""primary"";""no"";1729;""n..."
45208,"72;""retired"";""married"";""secondary"";""no"";5715;""..."
45209,"57;""blue-collar"";""married"";""secondary"";""no"";66..."


In [13]:
df1.describe

<bound method NDFrame.describe of       age;"job";"marital";"education";"default";"balance";"housing";"loan";"contact";"day";"month";"duration";"campaign";"pdays";"previous";"poutcome";"y"
0      58;"management";"married";"tertiary";"no";2143...                                                                                                  
1      44;"technician";"single";"secondary";"no";29;"...                                                                                                  
2      33;"entrepreneur";"married";"secondary";"no";2...                                                                                                  
3      47;"blue-collar";"married";"unknown";"no";1506...                                                                                                  
4      33;"unknown";"single";"unknown";"no";1;"no";"n...                                                                                                  
...                                 

In [14]:
sns.pairplot(df1)

ValueError: No variables found for grid columns.

In [ ]:
df1['y'].value_counts()

In [ ]:
count_no_sub = len(df1[df1['y']=="no"])
count_sub = len(df1[df1['y']=="yes"])

In [ ]:
(count_sub / (count_sub + count_no_sub))*100

### Percentage of Client Subscribed is 11.70 % in the current data set

In [ ]:
pd.crosstab(df1.job,df1.y).plot(kind='bar')
plt.title('Subscribed Frequency for Job Title')
plt.xlabel('Job')
plt.ylabel('Frequency of subscribtion')

The frequency of subscribtion depends a great deal on the job title. Thus, the job title can be a good predictor of the outcome variable.

In [ ]:
table=pd.crosstab(df1.marital,df1.y)
table.div(table.sum(1).astype(float), axis=0).plot(kind='bar', stacked=True)
plt.title('Stacked Bar Chart of Marital Status vs Subscribed')
plt.xlabel('Marital Status')
plt.ylabel('Proportion of Customers')

### The marital status seem a strong predictor for the outcome variable

In [ ]:
table=pd.crosstab(df1.education,df1.y)
table.div(table.sum(1).astype(float), axis=0).plot(kind='bar', stacked=True)
plt.title('Stacked Bar Chart of Education vs Subscribed')
plt.xlabel('Education')
plt.ylabel('Proportion of Customers')

### Education seem a strong predictor for the outcome variable

In [ ]:
table=pd.crosstab(df1.contact,df1.y)
table.div(table.sum(1).astype(float), axis=0).plot(kind='bar', stacked=True)
plt.title('Stacked Bar Chart of Contact vs Subscribed')
plt.xlabel('Contact')
plt.ylabel('Proportion of Customers')

### Contact does not seem a strong predictor for the outcome variable

In [ ]:
table=pd.crosstab(df1.poutcome,df1.y)
table.div(table.sum(1).astype(float), axis=0).plot(kind='bar', stacked=True)
plt.title('Stacked Bar Chart of Poutcome vs Subscribed')
plt.xlabel('Poutcome')
plt.ylabel('Proportion of Customers')

### Poutcome does not seem a strong predictor for the outcome variable

In [ ]:
df1.age.hist()
plt.title('Histogram of Age')
plt.xlabel('Age')
plt.ylabel('Frequency')

### Most of the customers are in age between 20 and 50 years

In [ ]:
pd.crosstab(df1.month,df1.y).plot(kind='bar')
plt.title('Subscribed Frequency for Month')
plt.xlabel('Month')
plt.ylabel('Frequency of Subscribed')

### Month might be a good predictor of the outcome variabl

In [ ]:
df1.day.hist()
plt.title('Histogram of Duration')
plt.xlabel('Duration')
plt.ylabel('Frequency')

In [ ]:
df1['housing'].value_counts()

### Data is somewhat evenly distributed on whether the client has House or not

In [ ]:
df1['loan'].value_counts()

### However majority of the client do not have loan

## __3 - Cleaning Data__ 

In [ ]:
df1.isnull().sum()

### Since there are no Null values in any column we don't have to create any exceptions

## __4 - Logistic Regression Model__ 

In [ ]:
df1 ['default'] = df1 ['default'].map({'yes': 1, 'no': 0})

In [ ]:
df1 ['housing'] = df1 ['housing'].map({'yes': 1, 'no': 0})

In [ ]:
df1 ['loan'] = df1 ['loan'].map({'yes': 1, 'no': 0})

In [ ]:
df1 ['y'] = df1 ['y'].map({'yes': 1, 'no': 0})

In [ ]:
df1 = pd.get_dummies(df1, columns=['job'])

In [ ]:
df1 = pd.get_dummies(df1, columns=['marital'])

In [ ]:
df1 = pd.get_dummies(df1, columns=['education'])

In [ ]:
df1 = pd.get_dummies(df1, columns=['month'])

In [ ]:
df1 = df1.drop(['contact', 'poutcome'], axis=1)

In [ ]:
X = df1.loc[:, df1.columns != 'y']
y = df1.loc[:, df1.columns == 'y']

In [ ]:
logreg = LogisticRegression()

In [ ]:
rfe = RFE(logreg, 20)
rfe = rfe.fit(X, y.values.ravel())
print(rfe.support_)
print(rfe.ranking_)

### As per Recursive Feature Elimination (RFE) analysis we can exclude all the variables which are False

In [ ]:
X = df1[['default', 'housing', 'loan', 'job_housemaid', 'job_retired', 'job_student', 'marital_married', 'education_primary', 'education_unknown', 'month_aug', 'month_dec', 'month_feb', 'month_jan', 'month_jul', 'month_jun', 'month_mar', 'month_may', 'month_nov', 'month_oct', 'month_sep']]
y = df1.loc[:, df1.columns == 'y']

In [ ]:
logit=sm.Logit(y,X)
result = logit.fit()

In [ ]:
result.summary()

#### All variables have significant p value

In [ ]:
logreg.fit(X, y)

In [ ]:
y_pred = logreg.predict(X)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X, y)))

In [ ]:
print(classification_report(y, y_pred))

In [ ]:
confusion_matrix(y, y_pred)

## __5 - Output Interpretation__ 

1 - Confusion Matrix  
The result is telling us that we have 39455+456 correct predictions and 4833+467 incorrect predictions.

2 - Accuracy == 84%   
Of the entire data set, 84% of the clients will subcribe